In [53]:
import numpy as np # for array, linear algebra
import pandas as pd # for data processing
import matplotlib.pyplot as plt # to crreate stat and visualize data
import matplotlib as mpl  # to visualize data
import seaborn as sns # to visualize data
import matplotlib.colors as mcolors # to visualize colors
import string # collection of alphabets, words or other characters
import re # regular expression support
import nltk 
from nltk.corpus import stopwords
from string import punctuation 
# import gensim # representing documents as semantic vectors

from nltk.corpus import sentiwordnet as swn
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/I570078/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/I570078/nltk_data...
[nltk_data] Downloading package punkt to /Users/I570078/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/I570078/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [54]:
pd.set_option('display.max_colwidth', None)

## Vader with Text-Pre-Edditing

In [55]:
df_sentiwordnet_without_stopwords = pd.read_csv('../../Data-Preperation/preprocessed_dataset.csv')

### Create a new column 'Ground_Truth_Label' based on the 'target' values

In [56]:
def map_target_to_label(target):
    if target == 4:
        return 'positive'
    elif target == 0:
        return 'negative'
    else:
        return None

In [57]:
df_sentiwordnet_without_stopwords['Ground_Truth_Label'] = df_sentiwordnet_without_stopwords['target'].apply(map_target_to_label)

In [58]:
df_sentiwordnet_without_stopwords

,target,Text,Ground_Truth_Label
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D",negative
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!,negative
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds,negative
3,0,my whole body feels itchy and like its on fire,negative
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.",negative
...,...,...,...
1599995,4,Just woke up. Having no school is the best feeling ever,positive
1599996,4,TheWDB.com - Very cool to hear old Walt interviews! ♫ http://blip.fm/~8bmta,positive
1599997,4,Are you ready for your MoJo Makeover? Ask me for details,positive
1599998,4,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur,positive


In [59]:
df_sentiwordnet_without_stopwords['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

### Dieser Classifier nutzt nur die Most-Common Wortbedeutung (Lemma)

In [60]:
def superNaiveSentiment(review):
    reviewPolarity = 0.0
    numExceptions = 0
    words = review.lower().split()
    
    for word in words:
        weight = 0.0
        synsets = list(swn.senti_synsets(word))
        if synsets:
            common_meaning = synsets[0]
            pos_score = common_meaning.pos_score()
            neg_score = common_meaning.neg_score()
            if pos_score > neg_score:
                weight += pos_score
            elif pos_score < neg_score:
                weight -= neg_score
        else:
            numExceptions += 1
        reviewPolarity += weight
    return reviewPolarity


In [61]:
df_sentiwordnet_without_stopwords

,target,Text,Ground_Truth_Label
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D",negative
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!,negative
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds,negative
3,0,my whole body feels itchy and like its on fire,negative
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.",negative
...,...,...,...
1599995,4,Just woke up. Having no school is the best feeling ever,positive
1599996,4,TheWDB.com - Very cool to hear old Walt interviews! ♫ http://blip.fm/~8bmta,positive
1599997,4,Are you ready for your MoJo Makeover? Ask me for details,positive
1599998,4,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur,positive


In [62]:
df_sentiwordnet_without_stopwords['Sentiwordnet_Classification'] = df_sentiwordnet_without_stopwords['Text'].apply(lambda x: 'positive' if superNaiveSentiment(x) >= 0 else 'negative')

In [63]:
df_sentiwordnet_without_stopwords

,target,Text,Ground_Truth_Label,Sentiwordnet_Classification
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D",negative,positive
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!,negative,negative
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds,negative,positive
3,0,my whole body feels itchy and like its on fire,negative,negative
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.",negative,negative
...,...,...,...,...
1599995,4,Just woke up. Having no school is the best feeling ever,positive,positive
1599996,4,TheWDB.com - Very cool to hear old Walt interviews! ♫ http://blip.fm/~8bmta,positive,positive
1599997,4,Are you ready for your MoJo Makeover? Ask me for details,positive,negative
1599998,4,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur,positive,positive


In [65]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_sentiwordnet_without_stopwords['Ground_Truth_Label'], df_sentiwordnet_without_stopwords['Sentiwordnet_Classification'])
print(f'Accuracy: {accuracy}')

ImportError: cannot import name 'accuracy_score' from 'sklearn' (/Users/I570078/Desktop/DHBW_Coding/Semester 4/Data Exploration/Github/venv/lib/python3.12/site-packages/sklearn/__init__.py)